In [5]:
pip install dash

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: C:\Users\jofre\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [6]:
pip install dash_daq

  Using cached dash_daq-0.5.0.tar.gz (642 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py install for dash_daq: started
  Running setup.py install for dash_daq: finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


  DEPRECATION: dash_daq is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559

[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: C:\Users\jofre\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [8]:
pip install dash_bootstrap_components

  Using cached dash_bootstrap_components-1.5.0-py3-none-any.whl (221 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: C:\Users\jofre\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [31]:
# On import les packages
from dash import Dash, html, dash_table, dcc
import pandas as pd
import plotly.express as px
import numpy as np
import dash_daq as daq
import dash_bootstrap_components as dbc


# On lit le jeu de données des décès en France ligne par ligne
lines = []
with open("data/deces-2023-m08.txt", "r") as file:
    lines = file.readlines()

# On charge le jeu de données pour pouvoir connaître la position géographique des villes en France
positions_geo = pd.read_csv('data/communes-departement-region.csv', usecols=[0, 1, 5, 6])


# On lit le jeu de données des positions des pays étrangers ligne par ligne
with open("data/positions-pays-étrangers.txt", "r") as file2:
    country_data = file2.readlines()

# On ajoute un 0 dans le cas où les codes on seulement 4 chiffres
positions_geo['code_commune_INSEE'] = positions_geo['code_commune_INSEE'].str.zfill(5)


# On initialise les listes vides pour chaque colonne du tableau
names = []
gender = []
dob = []
birthplace_code_list = []
birthplace = []
birthplace_details = []
year_list = []
dod = []
deathplace_code = []
death_cert_number = []
noms = []
prenoms = []
Age = []
same_places = []
deathplace = []

# Créez un dictionnaire pour stocker les correspondances deathplace_code_text -> birthplace_text
deathplace_mapping = {}

# Boucle pour extraire les données et trouver les correspondances
for line in lines:
    name = line[0:80].strip()
    nom, prenom = name.split('*')
    prenom = prenom.strip('/')
    gender_code = line[80:81]
    date_year = line[81:85]
    date_month = line[85:87]
    date_day = line[87:89]
    birthplace_code = line[89:94].strip()
    birthplace_text = line[94:124].strip()
    birthplace_details_text = line[124:154].strip()
    date_of_death_year = line[154:158]
    date_of_death_month = line[158:160]
    date_of_death_day = line[160:162]
    deathplace_code_text = line[162:167].strip()
    death_cert_number_text = line[167:176].strip()
    age = int(date_of_death_year) - int(date_year)

    # On doit convertir le code du genre en string
    if gender_code == '1':
        gender_text = "Masculin"
    elif gender_code == '2':
        gender_text = "Féminin"
    else:
        gender_text = "Inconnu"

    if birthplace_details_text == '':
        birthplace_details_text = "FRANCE"

    if deathplace_code_text == birthplace_code:
        new_place = (deathplace_code_text, birthplace_text)
        if new_place not in same_places:
            same_places.append(new_place)

        # Stockez la correspondance dans le dictionnaire
        deathplace_mapping[deathplace_code_text] = birthplace_text

    names.append(name)
    gender.append(gender_text)
    year_list.append(date_year)
    dob.append(date_day + "/" + date_month + "/" + date_year)
    birthplace_code_list.append(birthplace_code)
    birthplace.append(birthplace_text)
    birthplace_details.append(birthplace_details_text)
    dod.append(date_of_death_day + "/" + date_of_death_month + "/" + date_of_death_year)
    deathplace_code.append(deathplace_code_text)
    death_cert_number.append(death_cert_number_text)
    noms.append(nom)
    prenoms.append(prenom)
    Age.append(age)

# Boucle principale pour le traitement des lignes
for line in lines:
    deathplace_code_text = line[162:167].strip()
    deathplace_value = deathplace_mapping.get(deathplace_code_text, "NULL")
    deathplace.append(deathplace_value)

# Créez le tableau à partir des listes créées
data = {
    "Nom": noms,
    "Prenom(s)": prenoms,
    "Sex": gender,
    "Year": year_list,
    "Date of Birth": dob,
    "Age": Age,
    "Birthplace Code": birthplace_code_list,
    "Birthplace": birthplace,
    "Birthplace Details": birthplace_details,
    "Date of Death": dod,
    "Deathplace Code": deathplace_code,
    "Death Certificate Number": death_cert_number,
    "Death Place": deathplace,
}

# Créez le DataFrame avec pandas
df = pd.DataFrame(data)


# On crée un DataFrame avec toute l'information ensemble pour pouvoir comparer les Villes avec leur position géographique
# Realizar la fusión para las coordenadas de muerte
positions_geo.drop_duplicates(inplace=True)
merged_df_death = df.merge(positions_geo, left_on='Deathplace Code', right_on='code_commune_INSEE', how='left')
merged_df_death = merged_df_death.rename(columns={'longitude': 'longitude_death', 'latitude': 'latitude_death'})



In [33]:
merged_df_death[merged_df_death['Nom'] == 'COLL']

Nom            Prenom(s)       Sex  Year Date of Birth  Age  \
13906  COLL         ROBERT HENRI  Masculin  1934    02/04/1934   89   
16125  COLL                 ERIC  Masculin  1961    30/09/1961   62   
28947  COLL  GEORGES LOUIS DENIS  Masculin  1957    05/10/1957   66   
31839  COLL           ROSE MARIE   Féminin  1933    08/04/1933   90   
49808  COLL                ALAIN  Masculin  1960    14/01/1960   63   
78121  COLL              SOLANGE   Féminin  1947    21/11/1947   76   
79027  COLL               JOSEPH  Masculin  1934    18/05/1934   89   
88738  COLL    ANDRE ELIE JUSTIN  Masculin  1925    13/03/1925   98   

      Birthplace Code          Birthplace Birthplace Details Date of Death  \
13906           34225        PUISSERGUIER             FRANCE    03/07/2023   
16125           92352  DEPARTEMENT D'ORAN             FRANCE    02/07/2023   
28947           66129             ORTAFFA             FRANCE    20/07/2023   
31839           68224            MULHOUSE             FRANCE    16/07/2023   
49808           11206              LIMOUX             FRANCE    08/08/2023   
78121           64102             BAYONNE             FRANCE    10/08/2023   
79027           99134                                ESPAGNE    06/08/2023   
88738           81004                ALBI             FRANCE    31/07/2023   

      Deathplace Code Death Certificate Number  Death Place  \
13906           34070                        4         NULL   
16125           38185                      772     GRENOBLE   
28947           66136                     1693    PERPIGNAN   
31839           71263                      134      LOUHANS   
49808           11069                      776  CARCASSONNE   
78121           64102                      995      BAYONNE   
79027           66053                       48         NULL   
88738           81004                      750         ALBI   

      code_commune_INSEE nom_commune_postal  latitude_death  longitude_death  
13906              34070            CEBAZAN       43.405771         2.987535  
16125              38185           GRENOBLE       45.182122         5.721331  
28947              66136          PERPIGNAN       42.696595         2.899370  
31839              71263            LOUHANS       46.635188         5.238393  
49808              11069        CARCASSONNE       43.209380         2.343989  
78121              64102            BAYONNE       43.492225        -1.466077  
79027              66053          COLLIOURE       42.513011         3.072541  
88738              81004               ALBI       43.925821         2.146863

In [34]:
merged_df_birth = df.merge(positions_geo, left_on='Birthplace Code', right_on='code_commune_INSEE', how='left')
merged_df_birth = merged_df_birth.rename(columns={'longitude': 'longitude_birth', 'latitude': 'latitude_birth'})

In [35]:
merged_df_birth[merged_df_birth['Nom'] == 'COLL']

Nom            Prenom(s)       Sex  Year Date of Birth  Age  \
13906  COLL         ROBERT HENRI  Masculin  1934    02/04/1934   89   
16125  COLL                 ERIC  Masculin  1961    30/09/1961   62   
28947  COLL  GEORGES LOUIS DENIS  Masculin  1957    05/10/1957   66   
31839  COLL           ROSE MARIE   Féminin  1933    08/04/1933   90   
49808  COLL                ALAIN  Masculin  1960    14/01/1960   63   
78121  COLL              SOLANGE   Féminin  1947    21/11/1947   76   
79027  COLL               JOSEPH  Masculin  1934    18/05/1934   89   
88738  COLL    ANDRE ELIE JUSTIN  Masculin  1925    13/03/1925   98   

      Birthplace Code          Birthplace Birthplace Details Date of Death  \
13906           34225        PUISSERGUIER             FRANCE    03/07/2023   
16125           92352  DEPARTEMENT D'ORAN             FRANCE    02/07/2023   
28947           66129             ORTAFFA             FRANCE    20/07/2023   
31839           68224            MULHOUSE             FRANCE    16/07/2023   
49808           11206              LIMOUX             FRANCE    08/08/2023   
78121           64102             BAYONNE             FRANCE    10/08/2023   
79027           99134                                ESPAGNE    06/08/2023   
88738           81004                ALBI             FRANCE    31/07/2023   

      Deathplace Code Death Certificate Number  Death Place  \
13906           34070                        4         NULL   
16125           38185                      772     GRENOBLE   
28947           66136                     1693    PERPIGNAN   
31839           71263                      134      LOUHANS   
49808           11069                      776  CARCASSONNE   
78121           64102                      995      BAYONNE   
79027           66053                       48         NULL   
88738           81004                      750         ALBI   

      code_commune_INSEE nom_commune_postal  latitude_birth  longitude_birth  
13906              34225       PUISSERGUIER       43.377234         3.045905  
16125                NaN                NaN             NaN              NaN  
28947              66129            ORTAFFA       42.584347         2.920834  
31839              68224           MULHOUSE       47.749163         7.325700  
49808              11206             LIMOUX       43.049929         2.239588  
78121              64102            BAYONNE       43.492225        -1.466077  
79027                NaN                NaN             NaN              NaN  
88738              81004               ALBI       43.925821         2.146863

In [48]:

'''grouped_df_birth = merged_df_birth.groupby('Nom').agg({
    'longitude_birth': 'first',  # Utilisez 'first' pour obtenir le premier valeur non nulle.
    'latitude_birth': 'first',   # Utilisez 'first' pour obtenir le premier valeur non nulle.
}).reset_index()'''


"grouped_df_birth = merged_df_birth.groupby('Nom').agg({\n    'longitude_birth': 'first',  # Utilisez 'first' pour obtenir le premier valeur non nulle.\n    'latitude_birth': 'first',   # Utilisez 'first' pour obtenir le premier valeur non nulle.\n}).reset_index()"

In [49]:
'''grouped_df_birth[grouped_df_birth['Nom'] == 'COLL']'''

"grouped_df_birth[grouped_df_birth['Nom'] == 'COLL']"

In [59]:
# Fusionnez les données de décès avec les données de naissance en utilisant la colonne "Nom" comme clé
merged_df = merged_df_death.merge(merged_df_birth[['longitude_birth', 'latitude_birth']], left_index=True, right_index=True, how='left')
merged_df.loc[merged_df['Birthplace Details'] != 'FRANCE', ['latitude_birth', 'longitude_birth']] = np.nan

In [60]:
merged_df

Nom             Prenom(s)       Sex  Year Date of Birth  Age  \
0           CURTET                  RENE  Masculin  1938    12/08/1938   85   
1       STUPENENGO                  JEAN  Masculin  1927    24/05/1927   96   
2           GRUMEL       ETIENNE GABRIEL  Masculin  1924    16/05/1924   99   
3          CAZORLA             JOSEPHINE   Féminin  1931    23/11/1931   92   
4            DRAPE       ADRIENNE JEANNE   Féminin  1931    13/01/1931   92   
...            ...                   ...       ...   ...           ...  ...   
149442        WONG                 TEUMA   Féminin  1931    25/05/1931   84   
149443       AGMIR              BOUKHIAR  Masculin  1939    01/01/1939   84   
149444    DIERYCKX  DAMIEN FRANCK ALFRED  Masculin  1974    20/01/1974   48   
149445     REYNAUD     JEAN CLAUDE ROGER  Masculin  1946    12/07/1946   76   
149446   LATCHOUMY                         Féminin  1935    15/04/1935   88   

       Birthplace Code               Birthplace Birthplace Details  \
0                01053          BOURG-EN-BRESSE             FRANCE   
1                74077         CLARAFOND-ARCINE             FRANCE   
2                69382  LYON 2E  ARRONDISSEMENT             FRANCE   
3                99352           BENI-SAF, ORAN            ALGERIE   
4                64558             VILLEFRANQUE             FRANCE   
...                ...                      ...                ...   
149442           98732               NUKUTAVAKE             FRANCE   
149443           99350                TIMLIZINE              MAROC   
149444           59163                    CROIX             FRANCE   
149445           88169                     FAYS             FRANCE   
149446           99223             TRICHINAPOLY               INDE   

       Date of Death Deathplace Code Death Certificate Number  \
0         15/07/2023           01004                      151   
1         25/07/2023           01004                      154   
2         24/07/2023           01004                      155   
3         10/07/2023           01004                      148   
4         07/07/2023           01004                      147   
...              ...             ...                      ...   
149442    08/06/2015           99216                      310   
149443    29/08/2023           99350                      107   
149444    20/09/2022           99135                      609   
149445    11/07/2022           99126                      241   
149446    04/08/2023           99223                      294   

              Death Place code_commune_INSEE nom_commune_postal  \
0       AMBERIEU-EN-BUGEY              01004  AMBERIEU EN BUGEY   
1       AMBERIEU-EN-BUGEY              01004  AMBERIEU EN BUGEY   
2       AMBERIEU-EN-BUGEY              01004  AMBERIEU EN BUGEY   
3       AMBERIEU-EN-BUGEY              01004  AMBERIEU EN BUGEY   
4       AMBERIEU-EN-BUGEY              01004  AMBERIEU EN BUGEY   
...                   ...                ...                ...   
149442               NULL                NaN                NaN   
149443          TIMLIZINE                NaN                NaN   
149444              LEYDE                NaN                NaN   
149445         KHALANDRIO                NaN                NaN   
149446       TRICHINAPOLY                NaN                NaN   

        latitude_death  longitude_death  longitude_birth  latitude_birth  
0            45.960848         5.372926         5.246021       46.205152  
1            45.960848         5.372926         5.895805       46.074724  
2            45.960848         5.372926         4.826170       45.749256  
3            45.960848         5.372926              NaN             NaN  
4            45.960848         5.372926        -1.447668       43.442124  
...                ...              ...              ...             ...  
149442             NaN              NaN              NaN             NaN  
149443             NaN              NaN

In [61]:
merged_df[merged_df['Nom'] == 'COLL']

Nom            Prenom(s)       Sex  Year Date of Birth  Age  \
13906  COLL         ROBERT HENRI  Masculin  1934    02/04/1934   89   
16125  COLL                 ERIC  Masculin  1961    30/09/1961   62   
28947  COLL  GEORGES LOUIS DENIS  Masculin  1957    05/10/1957   66   
31839  COLL           ROSE MARIE   Féminin  1933    08/04/1933   90   
49808  COLL                ALAIN  Masculin  1960    14/01/1960   63   
78121  COLL              SOLANGE   Féminin  1947    21/11/1947   76   
79027  COLL               JOSEPH  Masculin  1934    18/05/1934   89   
88738  COLL    ANDRE ELIE JUSTIN  Masculin  1925    13/03/1925   98   

      Birthplace Code          Birthplace Birthplace Details Date of Death  \
13906           34225        PUISSERGUIER             FRANCE    03/07/2023   
16125           92352  DEPARTEMENT D'ORAN             FRANCE    02/07/2023   
28947           66129             ORTAFFA             FRANCE    20/07/2023   
31839           68224            MULHOUSE             FRANCE    16/07/2023   
49808           11206              LIMOUX             FRANCE    08/08/2023   
78121           64102             BAYONNE             FRANCE    10/08/2023   
79027           99134                                ESPAGNE    06/08/2023   
88738           81004                ALBI             FRANCE    31/07/2023   

      Deathplace Code Death Certificate Number  Death Place  \
13906           34070                        4         NULL   
16125           38185                      772     GRENOBLE   
28947           66136                     1693    PERPIGNAN   
31839           71263                      134      LOUHANS   
49808           11069                      776  CARCASSONNE   
78121           64102                      995      BAYONNE   
79027           66053                       48         NULL   
88738           81004                      750         ALBI   

      code_commune_INSEE nom_commune_postal  latitude_death  longitude_death  \
13906              34070            CEBAZAN       43.405771         2.987535   
16125              38185           GRENOBLE       45.182122         5.721331   
28947              66136          PERPIGNAN       42.696595         2.899370   
31839              71263            LOUHANS       46.635188         5.238393   
49808              11069        CARCASSONNE       43.209380         2.343989   
78121              64102            BAYONNE       43.492225        -1.466077   
79027              66053          COLLIOURE       42.513011         3.072541   
88738              81004               ALBI       43.925821         2.146863   

       longitude_birth  latitude_birth  
13906         3.045905       43.377234  
16125              NaN             NaN  
28947         2.920834       42.584347  
31839         7.325700       47.749163  
49808         2.239588       43.049929  
78121        -1.466077       43.492225  
79027              NaN             NaN  
88738         2.146863       43.925821

In [32]:



country_coordinates = {}

for line in country_data:
    parts = line.strip().split()
    if len(parts) == 3:
        country = parts[0].replace('*', ' ')
        latitude, longitude = parts[1], parts[2]
        country_coordinates[country] = (latitude, longitude)

def update_coordinates(row):
    country = row['Birthplace Details']
    if country in country_coordinates:
        latitude, longitude = country_coordinates[country]
        row['latitude_birth'] = latitude
        row['longitude_birth'] = longitude
    return row

merged_df = merged_df.apply(update_coordinates, axis=1)
merged_df['density'] = 1

# Initialisez l'application Dash
app = Dash(__name__, suppress_callback_exceptions=True, assets_folder='../assets', external_stylesheets=[dbc.themes.CYBORG])

theme = {
    'dark': True,
    'detail': '#007439',
    'primary': '#00EA64',
    'secondary': '#6E6E6E',
}


# App layout
def Accueil_layout():
    return html.Div(className='corpsacceuil',children=[
        html.H1(className='h1acc', children=["STATISTIQUES SUR LES DONNEES DE DECES"])
    ])

def page1_layout():
    return html.Div(className='corpslambda',children=[
        html.H1("Tableau df"),
        dash_table.DataTable(data=df.to_dict('records'), page_size=10),

        html.H1("Tableau df_merged"),
        dash_table.DataTable(data=merged_df.to_dict('records'), page_size=10),
    ])

In [29]:
merged_df[merged_df['Nom'] == 'COLL']

Nom            Prenom(s)       Sex  Year Date of Birth  Age  \
13906  COLL         ROBERT HENRI  Masculin  1934    02/04/1934   89   
16125  COLL                 ERIC  Masculin  1961    30/09/1961   62   
28947  COLL  GEORGES LOUIS DENIS  Masculin  1957    05/10/1957   66   
31839  COLL           ROSE MARIE   Féminin  1933    08/04/1933   90   
49808  COLL                ALAIN  Masculin  1960    14/01/1960   63   
78121  COLL              SOLANGE   Féminin  1947    21/11/1947   76   
79027  COLL               JOSEPH  Masculin  1934    18/05/1934   89   
88738  COLL    ANDRE ELIE JUSTIN  Masculin  1925    13/03/1925   98   

      Birthplace Code          Birthplace Birthplace Details Date of Death  \
13906           34225        PUISSERGUIER             FRANCE    03/07/2023   
16125           92352  DEPARTEMENT D'ORAN             FRANCE    02/07/2023   
28947           66129             ORTAFFA             FRANCE    20/07/2023   
31839           68224            MULHOUSE             FRANCE    16/07/2023   
49808           11206              LIMOUX             FRANCE    08/08/2023   
78121           64102             BAYONNE             FRANCE    10/08/2023   
79027           99134                                ESPAGNE    06/08/2023   
88738           81004                ALBI             FRANCE    31/07/2023   

      Deathplace Code Death Certificate Number  Death Place  \
13906           34070                        4         NULL   
16125           38185                      772     GRENOBLE   
28947           66136                     1693    PERPIGNAN   
31839           71263                      134      LOUHANS   
49808           11069                      776  CARCASSONNE   
78121           64102                      995      BAYONNE   
79027           66053                       48         NULL   
88738           81004                      750         ALBI   

      code_commune_INSEE nom_commune_postal  latitude_death  longitude_death  \
13906              34070            CEBAZAN       43.405771         2.987535   
16125              38185           GRENOBLE       45.182122         5.721331   
28947              66136          PERPIGNAN       42.696595         2.899370   
31839              71263            LOUHANS       46.635188         5.238393   
49808              11069        CARCASSONNE       43.209380         2.343989   
78121              64102            BAYONNE       43.492225        -1.466077   
79027              66053          COLLIOURE       42.513011         3.072541   
88738              81004               ALBI       43.925821         2.146863   

      longitude_birth latitude_birth  density  
13906        3.045905      43.377234        1  
16125        3.045905      43.377234        1  
28947        3.045905      43.377234        1  
31839        3.045905      43.377234        1  
49808        3.045905      43.377234        1  
78121        3.045905      43.377234        1  
79027         -3.7038        40.4168        1  
88738        3.045905      43.377234        1

In [27]:
myorigins_df = merged_df.copy()

# Extraction du premier prénom
myorigins_df['Prenom(s)'] = myorigins_df['Prenom(s)'].str.split().str[0]

# Filtrage des prénoms pour ne conserver que les chaînes de caractères
firstnames = myorigins_df['Prenom(s)'].dropna().unique()

# Filtrage des prénoms pour ne conserver que les chaînes de caractères
secondnames = myorigins_df['Nom'].dropna().unique()

# Création de firstname_options à partir des prénoms filtrés
firstname_options = [{'label': fstname, 'value': fstname} for fstname in firstnames]

# Tri de firstname_options
firstname_options.sort(key=lambda x: x['label'])

# Création de firstname_options à partir des prénoms filtrés
secondname_options = [{'label': sndname, 'value': sndname} for sndname in secondnames]

# Tri de firstname_options
secondname_options.sort(key=lambda x: x['label'])


In [30]:
myorigins_df[myorigins_df['Nom'] == 'COLL']

Nom Prenom(s)       Sex  Year Date of Birth  Age Birthplace Code  \
13906  COLL    ROBERT  Masculin  1934    02/04/1934   89           34225   
16125  COLL      ERIC  Masculin  1961    30/09/1961   62           92352   
28947  COLL   GEORGES  Masculin  1957    05/10/1957   66           66129   
31839  COLL      ROSE   Féminin  1933    08/04/1933   90           68224   
49808  COLL     ALAIN  Masculin  1960    14/01/1960   63           11206   
78121  COLL   SOLANGE   Féminin  1947    21/11/1947   76           64102   
79027  COLL    JOSEPH  Masculin  1934    18/05/1934   89           99134   
88738  COLL     ANDRE  Masculin  1925    13/03/1925   98           81004   

               Birthplace Birthplace Details Date of Death Deathplace Code  \
13906        PUISSERGUIER             FRANCE    03/07/2023           34070   
16125  DEPARTEMENT D'ORAN             FRANCE    02/07/2023           38185   
28947             ORTAFFA             FRANCE    20/07/2023           66136   
31839            MULHOUSE             FRANCE    16/07/2023           71263   
49808              LIMOUX             FRANCE    08/08/2023           11069   
78121             BAYONNE             FRANCE    10/08/2023           64102   
79027                                ESPAGNE    06/08/2023           66053   
88738                ALBI             FRANCE    31/07/2023           81004   

      Death Certificate Number  Death Place code_commune_INSEE  \
13906                        4         NULL              34070   
16125                      772     GRENOBLE              38185   
28947                     1693    PERPIGNAN              66136   
31839                      134      LOUHANS              71263   
49808                      776  CARCASSONNE              11069   
78121                      995      BAYONNE              64102   
79027                       48         NULL              66053   
88738                      750         ALBI              81004   

      nom_commune_postal  latitude_death  longitude_death longitude_birth  \
13906            CEBAZAN       43.405771         2.987535        3.045905   
16125           GRENOBLE       45.182122         5.721331        3.045905   
28947          PERPIGNAN       42.696595         2.899370        3.045905   
31839            LOUHANS       46.635188         5.238393        3.045905   
49808        CARCASSONNE       43.209380         2.343989        3.045905   
78121            BAYONNE       43.492225        -1.466077        3.045905   
79027          COLLIOURE       42.513011         3.072541         -3.7038   
88738               ALBI       43.925821         2.146863        3.045905   

      latitude_birth  density  
13906      43.377234        1  
16125      43.377234        1  
28947      43.377234        1  
31839      43.377234        1  
49808      43.377234        1  
78121      43.377234        1  
79027        40.4168        1  
88738      43.377234        1